In [ ]:
# 환경 설정 및 키 관리
import os
from getpass import getpass

# HTTP 요청 및 데이터 처리
import requests

# LangChain: 모델 및 메시지 관리
from langchain_openai import ChatOpenAI, OpenAIEmbeddings
from langchain_core.messages import HumanMessage
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.documents import Document

# LangChain: 텍스트 분할 및 벡터 스토어
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.vectorstores import FAISS

# 대화 기록 관리
from langchain_community.chat_message_histories import ChatMessageHistory
#날짜 시간 
from datetime import datetime

os.environ["OPENAI_API_KEY"] = getpass("OpenAI API key 입력: ")

# 2. 모델 초기화
from langchain_openai import ChatOpenAI
from langchain_core.messages import HumanMessage
model = ChatOpenAI(model="gpt-4o-mini")

# 3. 네이버 뉴스 API를 이용한 데이터 로드
import requests
from langchain_core.documents import Document

NAVER_NEWS_API_URL = "https://openapi.naver.com/v1/search/news.json"
headers = {
    "X-Naver-Client-Id": "8Jweb61zU8SEsEq2XxMM",
    "X-Naver-Client-Secret": "ThHHHbr1Je"
}

def get_news(query, display=10):
    params = {
        "query": query,
        "display": display,
        "sort": "sim"  # 유사도 기준 정렬
    }
    response = requests.get(NAVER_NEWS_API_URL, headers=headers, params=params)
    if response.status_code == 200:
        return response.json()
    else:
        print(f"뉴스 API 호출 오류: {response.status_code}")
        return None

# 4. 뉴스 데이터를 문서로 변환
class NaverNewsLoader:
    def __init__(self, news_data):
        self.news_data = news_data

    def load(self):
        documents = [
            Document(
                page_content=f"{item['title']}\n{item['description']}",
                metadata={"link": item['link'], "pubDate": item['pubDate']}
            )
            for item in self.news_data.get('items', [])
        ]
        return documents

# 5. 텍스트 청크 생성
from langchain.text_splitter import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(chunk_size=100, chunk_overlap=10)
def split_documents(documents):
    return text_splitter.split_documents(documents)

# 6. 임베딩 및 벡터 스토어 생성
from langchain_openai import OpenAIEmbeddings
from langchain_community.vectorstores import FAISS

embeddings = OpenAIEmbeddings(model="text-embedding-ada-002")

def create_vector_store(splits):
    return FAISS.from_documents(documents=splits, embedding=embeddings)

# 7. 대화 관리 및 RAG 체인
from langchain_core.prompts import ChatPromptTemplate
from langchain_community.chat_message_histories import ChatMessageHistory

class ContextToPrompt:
    def __init__(self, prompt_template):
        self.prompt_template = prompt_template

    def invoke(self, inputs):
        if isinstance(inputs, list):
            context_text = "\n".join([f"{doc.page_content}\n{doc.metadata['link']}" for doc in inputs])
        else:
            context_text = inputs
        return self.prompt_template.format_messages(context=context_text, question=inputs.get("question", ""))

class NewsRetriever:
    def __init__(self, vectorstore):
        self.vectorstore = vectorstore

    def retrieve(self, query):
        return self.vectorstore.as_retriever().get_relevant_documents(query)

prompt_template = ChatPromptTemplate.from_template(
    "다음 정보를 바탕으로 질문에 대한 답변을 작성하세요.\n\n"
    "뉴스 기사:\n{context}\n\n"
    "질문: {question}\n"
    "답변:"
)

chat_history = ChatMessageHistory()
def save_results(question, response, result_dir):
    
    timestamp = datetime.now().strftime("%Y-%m-%d_%H-%M-%S")  
    # 고정된 파일명 "prompt1.txt"에 덧붙이기
    prompt_file_path = os.path.join(result_dir, "Prompt", "prompt1.txt")
    
    # 디렉토리가 없으면 생성
    os.makedirs(os.path.dirname(prompt_file_path), exist_ok=True)

    # 질문을 고정된 파일에 덧붙이기
    with open(prompt_file_path, 'a', encoding='utf-8') as pf:
        pf.write(f"Question: {question}\n")
    
    # 고정된 파일명 "result1.txt"에 덧붙이기
  
    # 응답을 저장할 파일 경로 (타임스탬프 포함)
    result_file_path = os.path.join(result_dir, "result", f"result_{timestamp}.txt")
    
    # 디렉토리가 없으면 생성
    os.makedirs(os.path.dirname(result_file_path), exist_ok=True)

    # 응답을 고정된 파일에 덧붙이기
    with open(result_file_path, 'a', encoding='utf-8') as rf:
        rf.write(f"Response: {response}\n\n")
        
# 8. 메인 대화 루프
def main():
    result_dir = "C:\\Users\\dlswj\\Documents\\python\\quest"  # 결과 저장 디렉토리

    news_data = get_news("오늘의 뉴스")
    if not news_data:
        return
    
    loader = NaverNewsLoader(news_data)
    documents = loader.load()
    splits = split_documents(documents)
    vectorstore = create_vector_store(splits)
    retriever = NewsRetriever(vectorstore)

    while True:
        query = input("질문을 입력하세요: ")
        if query.strip().lower() in ["종료", "끝"]:
            print("대화를 종료합니다.")
            break

        # 검색 및 답변 생성
        response_docs = retriever.retrieve(query)
        prompt = ContextToPrompt(prompt_template).invoke({
            "context": response_docs,
            "question": query
        })
        response = model(prompt)

        # 대화 기록 관리
        chat_history.add_user_message(query)
        chat_history.add_ai_message(response.content)

        # 출력
        print(f"\n[질문] {query}")
        print(f"[답변] {response.content}\n")
        print(f"관련 뉴스 링크: {[doc.metadata['link'] for doc in response_docs]}")
        
        
        # 결과 저장
        save_results(query, response.content, result_dir)

if __name__ == "__main__":
    main()






[질문] 뉴스
[답변] 28일 주요 뉴스는 다음과 같습니다:

1. 더불어민주당이 대한민국 주식시장 활성화를 위한 태스크포스(TF)를 현장에서 진행합니다. (자세한 내용은 [뉴스핌](https://www.newspim.com/news/view/20241128000078)에서 확인할 수 있습니다.)
2. 오늘의 주요 뉴스와 관련된 프로그램이 오전 9시 30분에 방영됩니다. (관련 링크: [모닝와이드](https://n.news.naver.com/mnews/article/055/0001210097?sid=102))
3. 추가적인 뉴스와 뷰스를 전하는 내용이 월요일부터 금요일까지 매일 오전 9시 30분에 방송됩니다. (자세한 내용은 [한겨레](https://n.news.naver.com/mnews/article/028/0002717950?sid=110)에서 확인할 수 있습니다.)

더 많은 뉴스와 업데이트는 각 링크를 통해 확인하시기 바랍니다.

관련 뉴스 링크: ['https://n.news.naver.com/mnews/article/214/0001389327?sid=102', 'https://n.news.naver.com/mnews/article/055/0001210097?sid=102', 'https://n.news.naver.com/mnews/article/028/0002717950?sid=110', 'https://www.newspim.com/news/view/20241128000078']

[질문] 뉴스
[답변] 오늘의 주요 뉴스는 다음과 같습니다:

1. **더불어민주당 주식시장 활성화TF 현장** - 28일 오전 9시에 관련 프로그램이 방송될 예정입니다. 자세한 내용은 [뉴스핌TV](https://www.newspim.com/news/view/20241128000078)에서 확인하실 수 있습니다.
  
2. **모닝와이드 프로그램** - 오늘 아침 주요 뉴스에 대한 보도가 있었습니다. 관련 내용은 [여기](https://n.news.naver.com